##Goal:  Train a Naive Bayes model to classify future SMS messages as either spam or ham.

Steps:

1.  Convert the words ham and spam to a binary indicator variable(0/1)

2.  Convert the txt to a sparse matrix of TFIDF vectors

3.  Fit a Naive Bayes Classifier

4.  Measure your success using roc_auc_score



In [91]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

In [92]:
df= pd.read_csv("SMSSpamCollection",sep='\t', names=['spam', 'txt'])

In [93]:
df.spam.isnull().values.sum()

0

In [94]:
categorical_variables = ['spam']
for variable in categorical_variables:
    dummies = pd.get_dummies(df[variable], prefix=variable)
    df = pd.concat([df, dummies], axis=1)
    df.drop([variable], axis=1, inplace=True)

In [95]:
df.head()

,txt,spam_ham,spam_spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0


In [96]:
#TFIDF Vectorizer, just like before
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)

In [97]:
#in this case our dependent variable will be spam as 1 (ham) or 0 (spam)
y = df.spam_spam

In [98]:
#convert df.txt from text to features
X= vectorizer.fit_transform(df.txt)

In [99]:
#5572 observations x 8587 unique words.
print y.shape
print X.shape

(5572,)
(5572, 8587)


In [100]:
#Test Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [101]:
#we will train a naive_bayes classifier
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [102]:
#We can test our model's accuracy like this:

roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

0.98589322144123448